### 01. High Price Predict Model with Linear Regression

In [24]:
# 라이브러리
import pandas as pd
kospi = pd.read_csv('./KOSPI.csv')

# KOSPI에 상장된 종목 코드
kospi['종목코드'] = kospi['종목코드'].apply(lambda x : str(x).rjust(6, '0'))
code_list = kospi['종목코드'].unique()

# 라이브러리
import numpy as np

# 진행과정 Check
check = 0

# 결과 저장 데이터프레임
result_df = pd.DataFrame()

# 코드 별 진행
for code in code_list[:100] : 

    # Hyper Parameter
    tr_name = 'TR_SCHART'
    term = 'D'
    start_date = '20180510'   # 학습 기간 시작점
    end_date = '20200229'     # 학습 기간 종료점
    Lookup = '9999' 
    
    # url
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

    # url open
    from urllib.request import urlopen
    url_page = urlopen(url)

    # json 파일로 받아오기
    import json
    url_data = json.loads(url_page.read())

    # json을 데이터프레임으로 바꾸기
    import pandas as pd
    data = pd.DataFrame(url_data)
    
    # Data Column명 변경
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                    'rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    
    # 순서 뒤집기
    data = data[::-1]
    data = data.set_index('date')
    
    # 필요한 변수만 추출
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]

    # 만약에 데이터가 비어있다면 그 종목은 PASS
    if data["open"][0] == '' : 
        continue
    
    # 변수 속성 정수로 바꾸기
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
        
    # 거래량과 거래가격을 모두 로그 변환
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    
    # 각각의 피쳐들에 대해 
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    
    # 타겟 생성 : 다음날의 고가를 예측
    data['y'] = data['high'].shift(-1)
    
    # NaN이 들어가 있는 행 제거
    data = data.dropna()
    
    # Input과 Target을 분리
    X = data[data.columns[:-1]]
    y = data['y']
    
    # Linear Regression
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X, y)
    
    # Test Data 생성하기
    start_date, end_date = '20200301', '20200317'
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup
    url_page = urlopen(url)
    url_data = json.loads(url_page.read())
    data = pd.DataFrame(url_data)
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                    'rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    data = data[::-1]
    data = data.set_index('date')
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    data['y'] = data['high'].shift(-1)
    data = data.dropna()
    data = data[-1:]
    data = data[data.columns[:-1]]

    # 필요한것들
    result = dict({'종목' : code,
                   '어제 날짜' : data.index.values[0],
                   '어제 종가' : int(data['close'].values),
                   '오늘 고가' : int(round(reg.predict(data)[0])),
                   '오늘 고가 변동폭 예측' : 100*(int(round(reg.predict(data)[0]))-int(data['close'].values))/\
                                     int(data['close'].values)})
    result_df = result_df.append(result, ignore_index=True)
    
    # 각 종목별 결과 출력
    if check % 200 == 0 : 
        print('{:4} / {:4} ======> {:7.2f}% 완료'.format(check, 2144, 100*check/2144))
    check += 1

   0 / 2144 ======>    0.00% 완료


### 02. 예측 결과

In [25]:
result_df = result_df[['어제 날짜', '종목', '어제 종가', '오늘 고가', '오늘 고가 변동폭 예측']]

In [26]:
result_df['오늘 고가 변동폭 예측'] = result_df['오늘 고가 변동폭 예측'].apply(lambda x : round(x,2))

In [27]:
result_df

,어제 날짜,종목,어제 종가,오늘 고가,오늘 고가 변동폭 예측
0,20200316,060310,1890.0,1996.0,5.61
1,20200316,095570,3415.0,3453.0,1.11
2,20200316,068400,7550.0,7900.0,4.64
3,20200316,006840,18800.0,18285.0,-2.74
4,20200316,054620,6330.0,6467.0,2.16
...,...,...,...,...,...
94,20200316,092230,40600.0,40902.0,0.74
95,20200316,000040,633.0,565.0,-10.74
96,20200316,044450,6650.0,6801.0,2.27
97,20200316,030210,1950.0,2062.0,5.74


### 03. 내일 고가 변동폭이 가장 높은 TOP 10

In [28]:
result_df = result_df.sort_values(by='오늘 고가 변동폭 예측', ascending=False).set_index('종목')

In [29]:
result_df = result_df[result_df['오늘 고가 변동폭 예측'] < 30.00]

In [30]:
result_df[:10]

,어제 날짜,어제 종가,오늘 고가,오늘 고가 변동폭 예측
종목,,,,
032860,20200316,270.0,334.0,23.70
058400,20200316,690.0,823.0,19.28
297570,20200316,1550.0,1787.0,15.29
119650,20200316,3490.0,3945.0,13.04
306620,20200316,1535.0,1730.0,12.70
037370,20200316,6020.0,6613.0,9.85
065770,20200316,1680.0,1842.0,9.64
082740,20200316,2550.0,2783.0,9.14
045890,20200316,2420.0,2640.0,9.09


In [32]:
# 특정 종목 결과 확인
# result_df.loc['145020']

### 04. 결과 송신

In [9]:
# # 딕셔너리로 저장하기
# pred = dict({"model" : "psh_lr_high",
#              "seed" : 1234,
#              "type" : "high",
#              "code" : code,
#              "date" : "20-03-16 00:00:00",
#              "prdate" : 0,
#              "value" : int(round(reg.predict(data)[0]))})
    
# # 제출 리스트에 추가하기
# submission.append(pred)

In [10]:
# import requests
# import json

# def send_predicts(predict):
#     payload = json.dumps(predict)
#     print(payload)
#     url = "https://lab.donutz.co/tensors"
#     headers = {'Content-Type': 'application/json; charset=utf-8'}
#     #response = requests.get(url, params=paramDict)
#     response = requests.post(url, data=payload, headers=headers, verify=False)
#     code = response.status_code
#     res = response.text
#     return (code, res)

# predict = submission
# print(send_predicts(predict))